In [1]:
!pip install keras
!pip install tensorflow

You should consider upgrading via the 'c:\users\deekshitha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\deekshitha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
#!pip install opencv-python
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD
import cv2

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 5
epochs = 5

In [4]:
emotion_data = pd.read_csv('fer2013.csv', engine="python")
print(emotion_data)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [5]:
emotion_data.dtypes

emotion     int64
pixels     object
Usage      object
dtype: object

In [6]:
X_train = []
y_train = []
X_test = []
y_test = []
for index, row in emotion_data.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])

In [7]:
from keras.utils import np_utils 
X_train = np.array(X_train, 'float32')
y_train = np.array(y_train, 'float32')
X_test = np.array(X_test, 'float32')
y_test = np.array(y_test, 'float32')

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_train = X_train.astype('float32')
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)
X_test = X_test.astype('float32')

y_train= np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)
a=print(len(X_train))
b=print(len(X_test))
c=print(len(y_train))
d=print(len(y_test))

28709
3589
28709
3589


In [8]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(48,48,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), padding='same'))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), padding='same'))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), padding='same'))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), padding='same'))


model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), padding='same'))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [9]:
from keras.preprocessing.image import ImageDataGenerator
import keras.optimizers
gen = ImageDataGenerator()
train_generator = gen.flow(X_train, y_train, batch_size=batch_size)


In [10]:
model.compile(loss='categorical_crossentropy'
    , optimizer=keras.optimizers.Adam()
    , metrics=['accuracy']
)


In [11]:
fit = True

if fit == True:
    #model.fit_generator(x_train, y_train, epochs=epochs) #train for all trainset
    model.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=epochs) #train for randomly selected one
else:
	model.load_weights('/data/facial_expression_model_weights.h5') #load weights

c:\users\deekshitha\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp8soc2dgh.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp8soc2dgh.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
5/5 [==============================] - 47s 647ms/step - loss: 1.9426 - accuracy: 0.3650
Epoch 2/5
5/5 [==============================] - 3s 627ms/step - loss: 1.9412 - accuracy: 0.1289
Epoch 3/5
5/5 [==============================] - 3s 696ms/step - loss: 1.8848 - accuracy: 0.1967
Epoch 4/5
5/5 [==============================] - 2s 497ms/step - loss: 1.8472 - accuracy: 0.2494
Epoch 5

In [12]:
#overall evaluation
score = model.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', 100*score[1])

113/113 [==============================] - 9s 50ms/step - loss: 1.8607 - accuracy: 0.2494
Test loss: 1.8607465028762817
Test accuracy: 24.937307834625244


In [13]:
loss_and_metrics = model.evaluate(X_test,y_test)
print(loss_and_metrics)

113/113 [==============================] - 6s 49ms/step - loss: 1.8607 - accuracy: 0.2494
[1.8607465028762817, 0.24937307834625244]


In [14]:

model_json = model.to_json()
with open("model.json", "w") as json_file:
     json_file.write(model_json)
     model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [15]:
#!pip install mtcnn
from tensorflow.keras import models
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN


In [16]:

model.load_weights('model.h5')
model.summary()
cv2.ocl.setUseOpenCL(False)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 50, 50, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 64)        640       
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 64)          36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 64)          0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 128)         7

In [17]:
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: 'Neutral'}

In [18]:
import time

In [29]:
cap = cv2.VideoCapture(0)
codec = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('CAPTURE.avi', codec, 30, (640, 480))
while (True):
        # Find haar cascade to draw bounding box around face
        ret, frame = cap.read()
        output.write(frame)
        if not ret:
            break
        facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
            prediction = model.predict(cropped_img)
            maxindex = int(np.argmax(prediction))
            cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            #emotion_dict[maxindex]

        cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
        cv2.waitKey(10000)
          break
        '''if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            break'''
           
cap.release()
output.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [24]:
print(emotion_dict[maxindex])

Happy


# based on face emotion predicting songs

In [27]:
!pip install librosa
!pip install seaborn
import pandas as pd
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
import seaborn as sns
#!pip install playsound
from playsound import playsound

#from IPython.display import *

You should consider upgrading via the 'c:\users\deekshitha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\deekshitha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'playsound'

In [ ]:
feedback_df=pd.read_csv("emotify data.csv",sep=',')
feedback_df.columns = ["track id","genre","amazement","solemnity","tenderness","nostalgia","calmness","power","joyful_activation","tension","sadness","mood","liked","disliked","age","gender","mother tongue"]
feedback_df

In [48]:
class ranking_each_song_based_on_emotion():
    def create_new_columns(self):
        self.rank_list_amazeme=[]
        self.rank_list_solemnity=[]
        self.rank_list_tenderness=[]
        self.rank_list_nostalgia=[]
        self.rank_list_calmness=[]
        self.rank_list_power=[]
        self.rank_list_joyful_activation=[]
        self.rank_list_tension=[]
        self.rank_list_sadness=[]
        
        
    def classic_rank(self):
        count=100
        i=1
        while(i<=count):
            genre=feedback_df.loc[(feedback_df['genre'] == "classical") & (feedback_df['track id'] == i)]
            emotion_rank=genre.sum(axis = 0, skipna = True)
            amazeme_rank=emotion_rank[2]/len(genre)
            temp_list_amazeme=[amazeme_rank]*len(genre)
            self.rank_list_amazeme=self.rank_list_amazeme+temp_list_amazeme
            
            solemnity_rank=emotion_rank[3]/len(genre)
            temp_list_solemnity=[solemnity_rank]*len(genre)
            self.rank_list_solemnity=self.rank_list_solemnity+temp_list_solemnity
            
            tenderness_rank=emotion_rank[4]/len(genre)
            temp_list_tenderness=[tenderness_rank]*len(genre)
            self.rank_list_tenderness=self.rank_list_tenderness+temp_list_tenderness
            
            nostalgia_rank=emotion_rank[5]/len(genre)
            temp_list_nostalgia=[nostalgia_rank]*len(genre)
            self.rank_list_nostalgia=self.rank_list_nostalgia+temp_list_nostalgia
            
            calmness_rank=emotion_rank[6]/len(genre)
            temp_list_calmness=[calmness_rank]*len(genre)
            self.rank_list_calmness=self.rank_list_calmness+temp_list_calmness
            
            power_rank=emotion_rank[7]/len(genre)
            temp_list_power=[power_rank]*len(genre)
            self.rank_list_power=self.rank_list_power+temp_list_power
            
            joyful_activation_rank=emotion_rank[8]/len(genre)
            temp_list_joyful_activation=[joyful_activation_rank]*len(genre)
            self.rank_list_joyful_activation=self.rank_list_joyful_activation+temp_list_joyful_activation
            
            tension_rank=emotion_rank[9]/len(genre)
            temp_list_tension=[tension_rank]*len(genre)
            self.rank_list_tension=self.rank_list_tension+temp_list_tension
            
            
            sadness_rank=emotion_rank[10]/len(genre)
            temp_list_sadness=[sadness_rank]*len(genre)
            self.rank_list_sadness=self.rank_list_sadness+temp_list_sadness
            i+=1
    
    def rock_rank(self):
        count=200
        i=101
        while(i<=count):
            genre=feedback_df.loc[(feedback_df['genre'] == "rock") & (feedback_df['track id'] == i)]
            emotion_rank=genre.sum(axis = 0, skipna = True)
            amazeme_rank=emotion_rank[2]/len(genre)
            temp_list_amazeme=[amazeme_rank]*len(genre)
            self.rank_list_amazeme=self.rank_list_amazeme+temp_list_amazeme
            
            solemnity_rank=emotion_rank[3]/len(genre)
            temp_list_solemnity=[solemnity_rank]*len(genre)
            self.rank_list_solemnity=self.rank_list_solemnity+temp_list_solemnity
            
            tenderness_rank=emotion_rank[4]/len(genre)
            temp_list_tenderness=[tenderness_rank]*len(genre)
            self.rank_list_tenderness=self.rank_list_tenderness+temp_list_tenderness
            
            nostalgia_rank=emotion_rank[5]/len(genre)
            temp_list_nostalgia=[nostalgia_rank]*len(genre)
            self.rank_list_nostalgia=self.rank_list_nostalgia+temp_list_nostalgia
            
            calmness_rank=emotion_rank[6]/len(genre)
            temp_list_calmness=[calmness_rank]*len(genre)
            self.rank_list_calmness=self.rank_list_calmness+temp_list_calmness
            
            power_rank=emotion_rank[7]/len(genre)
            temp_list_power=[power_rank]*len(genre)
            self.rank_list_power=self.rank_list_power+temp_list_power
            
            joyful_activation_rank=emotion_rank[8]/len(genre)
            temp_list_joyful_activation=[joyful_activation_rank]*len(genre)
            self.rank_list_joyful_activation=self.rank_list_joyful_activation+temp_list_joyful_activation
            
            tension_rank=emotion_rank[9]/len(genre)
            temp_list_tension=[tension_rank]*len(genre)
            self.rank_list_tension=self.rank_list_tension+temp_list_tension
            
            
            sadness_rank=emotion_rank[10]/len(genre)
            temp_list_sadness=[sadness_rank]*len(genre)
            self.rank_list_sadness=self.rank_list_sadness+temp_list_sadness
            i+=1
    
    def electronic_rank(self):
        count=300
        i=201
        while(i<=count):
            genre=feedback_df.loc[(feedback_df['genre'] == "electronic") & (feedback_df['track id'] == i)]
            emotion_rank=genre.sum(axis = 0, skipna = True)
            amazeme_rank=emotion_rank[2]/len(genre)
            temp_list_amazeme=[amazeme_rank]*len(genre)
            self.rank_list_amazeme=self.rank_list_amazeme+temp_list_amazeme
            
            solemnity_rank=emotion_rank[3]/len(genre)
            temp_list_solemnity=[solemnity_rank]*len(genre)
            self.rank_list_solemnity=self.rank_list_solemnity+temp_list_solemnity
            
            tenderness_rank=emotion_rank[4]/len(genre)
            temp_list_tenderness=[tenderness_rank]*len(genre)
            self.rank_list_tenderness=self.rank_list_tenderness+temp_list_tenderness
            
            nostalgia_rank=emotion_rank[5]/len(genre)
            temp_list_nostalgia=[nostalgia_rank]*len(genre)
            self.rank_list_nostalgia=self.rank_list_nostalgia+temp_list_nostalgia
            
            calmness_rank=emotion_rank[6]/len(genre)
            temp_list_calmness=[calmness_rank]*len(genre)
            self.rank_list_calmness=self.rank_list_calmness+temp_list_calmness
            
            power_rank=emotion_rank[7]/len(genre)
            temp_list_power=[power_rank]*len(genre)
            self.rank_list_power=self.rank_list_power+temp_list_power
            
            joyful_activation_rank=emotion_rank[8]/len(genre)
            temp_list_joyful_activation=[joyful_activation_rank]*len(genre)
            self.rank_list_joyful_activation=self.rank_list_joyful_activation+temp_list_joyful_activation
            
            tension_rank=emotion_rank[9]/len(genre)
            temp_list_tension=[tension_rank]*len(genre)
            self.rank_list_tension=self.rank_list_tension+temp_list_tension
            
            
            sadness_rank=emotion_rank[10]/len(genre)
            temp_list_sadness=[sadness_rank]*len(genre)
            self.rank_list_sadness=self.rank_list_sadness+temp_list_sadness
            i+=1
    
    
    def pop_rank(self):
        count=400
        i=301
        while(i<=count):
            genre=feedback_df.loc[(feedback_df['genre'] == "pop") & (feedback_df['track id'] == i)]
            emotion_rank=genre.sum(axis = 0, skipna = True)
            amazeme_rank=emotion_rank[2]/len(genre)
            temp_list_amazeme=[amazeme_rank]*len(genre)
            self.rank_list_amazeme=self.rank_list_amazeme+temp_list_amazeme
            
            solemnity_rank=emotion_rank[3]/len(genre)
            temp_list_solemnity=[solemnity_rank]*len(genre)
            self.rank_list_solemnity=self.rank_list_solemnity+temp_list_solemnity
            
            tenderness_rank=emotion_rank[4]/len(genre)
            temp_list_tenderness=[tenderness_rank]*len(genre)
            self.rank_list_tenderness=self.rank_list_tenderness+temp_list_tenderness
            
            nostalgia_rank=emotion_rank[5]/len(genre)
            temp_list_nostalgia=[nostalgia_rank]*len(genre)
            self.rank_list_nostalgia=self.rank_list_nostalgia+temp_list_nostalgia
            
            calmness_rank=emotion_rank[6]/len(genre)
            temp_list_calmness=[calmness_rank]*len(genre)
            self.rank_list_calmness=self.rank_list_calmness+temp_list_calmness
            
            power_rank=emotion_rank[7]/len(genre)
            temp_list_power=[power_rank]*len(genre)
            self.rank_list_power=self.rank_list_power+temp_list_power
            
            joyful_activation_rank=emotion_rank[8]/len(genre)
            temp_list_joyful_activation=[joyful_activation_rank]*len(genre)
            self.rank_list_joyful_activation=self.rank_list_joyful_activation+temp_list_joyful_activation
            
            tension_rank=emotion_rank[9]/len(genre)
            temp_list_tension=[tension_rank]*len(genre)
            self.rank_list_tension=self.rank_list_tension+temp_list_tension
            
            
            sadness_rank=emotion_rank[10]/len(genre)
            temp_list_sadness=[sadness_rank]*len(genre)
            self.rank_list_sadness=self.rank_list_sadness+temp_list_sadness
            i+=1
    
    def add_new_columns(self):
        feedback_df["amazement_rank"]=self.rank_list_amazeme
        feedback_df["solemnity_rank"]=self.rank_list_solemnity
        feedback_df["tenderness_rank"]=self.rank_list_tenderness
        feedback_df["nostalgia_rank"]=self.rank_list_nostalgia
        feedback_df["calmness_rank"]=self.rank_list_calmness
        feedback_df["power_rank"]=self.rank_list_power
        feedback_df["joyful_activation_rank"]=self.rank_list_joyful_activation
        feedback_df["tension_rank"]=self.rank_list_tension
        feedback_df["sadness_rank"]=self.rank_list_sadness
        
    

In [49]:
amazement_rank=[0]*len(feedback_df)
feedback_df['amazement_rank']=amazement_rank
feedback_df['solemnity_rank']=amazement_rank
feedback_df['tenderness_rank']=amazement_rank
feedback_df['nostalgia_rank']=amazement_rank
feedback_df['calmness_rank']=amazement_rank
feedback_df['power_rank']=amazement_rank
feedback_df['joyful_activation_rank']=amazement_rank
feedback_df['tension_rank']=amazement_rank
feedback_df['sadness_rank']=amazement_rank


is_model = ranking_each_song_based_on_emotion()
is_model.create_new_columns()
is_model.classic_rank()
is_model.rock_rank()
is_model.electronic_rank()
is_model.pop_rank()
is_model.add_new_columns()
feedback_df

,track id,genre,amazement,solemnity,tenderness,nostalgia,calmness,power,joyful_activation,tension,...,mother tongue,amazement_rank,solemnity_rank,tenderness_rank,nostalgia_rank,calmness_rank,power_rank,joyful_activation_rank,tension_rank,sadness_rank
0,1,classical,0,1,0,0,0,0,1,1,...,English,0.145833,0.333333,0.208333,0.291667,0.625000,0.020833,0.083333,0.0625,0.312500
1,1,classical,0,0,0,1,0,0,0,0,...,Dutch,0.145833,0.333333,0.208333,0.291667,0.625000,0.020833,0.083333,0.0625,0.312500
2,1,classical,0,0,0,1,0,0,0,0,...,English,0.145833,0.333333,0.208333,0.291667,0.625000,0.020833,0.083333,0.0625,0.312500
3,1,classical,0,0,0,0,1,0,0,0,...,Spanish,0.145833,0.333333,0.208333,0.291667,0.625000,0.020833,0.083333,0.0625,0.312500
4,1,classical,0,0,0,1,1,0,0,0,...,English,0.145833,0.333333,0.208333,0.291667,0.625000,0.020833,0.083333,0.0625,0.312500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8402,400,pop,1,1,0,0,0,0,1,0,...,Russian,0.250000,0.166667,0.083333,0.333333,0.083333,0.000000,0.250000,0.5000,0.416667
8403,400,pop,0,0,0,1,0,0,1,0,...,Russian,0.250000,0.166667,0.083333,0.333333,0.083333,0.000000,0.250000,0.5000,0.416667
8404,400,pop,0,0,0,0,0,0,0,1,...,Polish,0.250000,0.166667,0.083333,0.333333,0.083333,0.000000,0.250000,0.5000,0.416667
8405,400,pop,1,0,0,0,0,0,1,1,...,French,0.250000,0.166667,0.083333,0.333333,0.083333,0.000000,0.250000,0.5000,0.416667


In [50]:
rowdata_classical=feedback_df.loc[[0,48,95,141,183,227,274,321,367,411,458,505,556,603,652,696,740,779,830,877,922,967,1007,1053,1102,1148,1192,1232,1286,1334,1380,1427,1471,1506,1531,1548,1565,1583,1600,1619,1636,1655,1673,1689,1706,1723,1740,1757,1774,1791,1808,1826,1844,1861,1879,1897,1915,1933,1950,1967,1984,2002,2019,2036,2053,2070,2087,2104,2122,2160,2178,2195,2212,2229,2247,2264,2281,2298,2315,2332,2349,2365,2381,2398,2415,2432,2450,2467,2484,2501,2518,2535,2552,2569,2586,2603,2620,2637,2654,2671],:]
rowdata_classical=rowdata_classical.drop(['amazement','solemnity','tenderness','nostalgia','calmness','power','joyful_activation','tension','sadness','mood','liked','disliked','age','gender','mother tongue'],axis=1)
rowdata_classical.tail(10)

,track id,genre,amazement_rank,solemnity_rank,tenderness_rank,nostalgia_rank,calmness_rank,power_rank,joyful_activation_rank,tension_rank,sadness_rank
2518,91,classical,0.000000,0.411765,0.176471,0.294118,0.352941,0.058824,0.000000,0.058824,0.588235
2535,92,classical,0.117647,0.411765,0.176471,0.352941,0.529412,0.000000,0.117647,0.058824,0.117647
2552,93,classical,0.117647,0.000000,0.352941,0.235294,0.882353,0.000000,0.176471,0.000000,0.117647
2569,94,classical,0.294118,0.588235,0.000000,0.058824,0.352941,0.117647,0.294118,0.235294,0.176471
2586,95,classical,0.294118,0.176471,0.235294,0.176471,0.117647,0.117647,0.705882,0.058824,0.117647
2603,96,classical,0.058824,0.176471,0.352941,0.294118,0.470588,0.058824,0.352941,0.058824,0.058824
2620,97,classical,0.117647,0.470588,0.058824,0.176471,0.235294,0.000000,0.000000,0.294118,0.352941
2637,98,classical,0.058824,0.411765,0.235294,0.352941,0.588235,0.000000,0.000000,0.058824,0.352941
2654,99,classical,0.117647,0.117647,0.411765,0.235294,0.882353,0.117647,0.176471,0.058824,0.176471
2671,100,classical,0.117647,0.470588,0.000000,0.117647,0.176471,0.529412,0.117647,0.294118,0.117647


In [51]:
rowdata_rock=feedback_df.loc[[2688,2738,2789,2837,2882,2926,2970,3014,3052,3098,3144,3194,3246,3292,3344,3390,3439,3484,3522,3547,3573,3593,3612,3629,3645,3659,3673,3687,3702,3716,3730,3744,3758,3772,3787,3801,3815,3829,3843,3857,3870,3885,3899,3913,3927,3941,3955,3969,3983,3997,4011,4024,4038,4052,4066,4080,4094,4108,4122,4136,4150,4164,4178,4192,4206,4220,4234,4248,4261,4274,4287,4301,4315,4329,4342,4356,4389,4403,4417,4431,4445,4459,4473,4487,4501,4516,4529,4543,4557,4572,4586,4599,4613,4626,4642,4656,4670,4684,4698,4712],:]
rowdata_rock=rowdata_rock.drop(['amazement','solemnity','tenderness','nostalgia','calmness','power','joyful_activation','tension','sadness','mood','liked','disliked','age','gender','mother tongue'],axis=1)
rowdata_rock.tail(10)

,track id,genre,amazement_rank,solemnity_rank,tenderness_rank,nostalgia_rank,calmness_rank,power_rank,joyful_activation_rank,tension_rank,sadness_rank
4586,191,rock,0.153846,0.230769,0.461538,0.461538,0.461538,0.076923,0.153846,0.076923,0.153846
4599,192,rock,0.000000,0.142857,0.428571,0.428571,0.428571,0.142857,0.214286,0.142857,0.142857
4613,193,rock,0.000000,0.153846,0.461538,0.538462,0.461538,0.000000,0.000000,0.076923,0.615385
4626,194,rock,0.125000,0.062500,0.250000,0.250000,0.312500,0.187500,0.375000,0.000000,0.312500
4642,195,rock,0.000000,0.142857,0.500000,0.642857,0.642857,0.000000,0.000000,0.000000,0.428571
4656,196,rock,0.214286,0.142857,0.000000,0.500000,0.428571,0.071429,0.285714,0.142857,0.285714
4670,197,rock,0.142857,0.142857,0.071429,0.142857,0.071429,0.285714,0.428571,0.214286,0.071429
4684,198,rock,0.214286,0.000000,0.071429,0.142857,0.071429,0.428571,0.714286,0.000000,0.000000
4698,199,rock,0.357143,0.142857,0.071429,0.214286,0.000000,0.642857,0.285714,0.071429,0.071429
4712,200,rock,0.214286,0.000000,0.285714,0.214286,0.142857,0.071429,0.214286,0.071429,0.071429


In [52]:
rowdata_electronic=feedback_df.loc[[4726,4776,4826,4870,4920,4959,5007,5066,5113,5165,5210,5268,5321,5379,5435,5492,5535,5569,5598,5609,5636,5658,5680,5691,5702,5720,5733,5744,5755,5766,5777,5789,5800,5811,5822,5833,5844,5855,5866,5877,5888,5899,5910,5921,5932,5943,5954,5965,5976,5987,5999,6010,6021,6032,6043,6055,6066,6077,6088,6099,6111,6122,6133,6144,6155,6166,6177,6188,6199,6210,6221,6232,6243,6254,6265,6276,6287,6298,6309,6320,6331,6342,6353,6365,6376,6387,6398,6409,6420,6431,6442,6453,6464,6475,6486,6497,6508,6519,6530,6541],:]
rowdata_electronic=rowdata_electronic.drop(['amazement','solemnity','tenderness','nostalgia','calmness','power','joyful_activation','tension','sadness','mood','liked','disliked','age','gender','mother tongue'],axis=1)
rowdata_electronic.tail(10)

,track id,genre,amazement_rank,solemnity_rank,tenderness_rank,nostalgia_rank,calmness_rank,power_rank,joyful_activation_rank,tension_rank,sadness_rank
6442,291,electronic,0.000000,0.545455,0.181818,0.272727,0.545455,0.181818,0.090909,0.272727,0.090909
6453,292,electronic,0.181818,0.181818,0.000000,0.000000,0.272727,0.363636,0.545455,0.363636,0.000000
6464,293,electronic,0.090909,0.181818,0.090909,0.090909,0.090909,0.636364,0.363636,0.363636,0.000000
6475,294,electronic,0.272727,0.181818,0.181818,0.090909,0.363636,0.090909,0.181818,0.090909,0.090909
6486,295,electronic,0.090909,0.181818,0.090909,0.181818,0.545455,0.000000,0.000000,0.181818,0.272727
6497,296,electronic,0.000000,0.090909,0.272727,0.272727,0.454545,0.181818,0.545455,0.272727,0.090909
6508,297,electronic,0.000000,0.454545,0.000000,0.000000,0.181818,0.454545,0.090909,0.363636,0.363636
6519,298,electronic,0.181818,0.181818,0.090909,0.181818,0.363636,0.181818,0.545455,0.090909,0.272727
6530,299,electronic,0.090909,0.545455,0.090909,0.090909,0.272727,0.090909,0.090909,0.363636,0.272727
6541,300,electronic,0.090909,0.000000,0.090909,0.090909,0.090909,0.454545,0.454545,0.363636,0.090909


In [53]:
rowdata_pop=feedback_df.loc[[6552,6605,6650,6700,6746,6800,6848,6890,6941,6988,7033,7080,7132,7189,7238,7282,7323,7359,7387,7406,7425,7440,7453,7465,7477,7489,7501,7513,7528,7540,7552,7564,7575,7587,7598,7610,7624,7635,7647,7659,7671,7683,7695,7707,7719,7732,7743,7755,7768,7780,7792,7804,7816,7828,7840,7852,7865,7877,7889,7901,7913,7925,7936,7948,7960,7973,7985,7997,8009,8021,8049,8061,8073,8085,8097,8109,8121,8133,8144,8156,8168,8181,8193,8205,8217,8229,8240,8252,8264,8276,8288,8300,8312,8324,8337,8349,8361,8372,8383,8395],:]
rowdata_pop=rowdata_pop.drop(['amazement','solemnity','tenderness','nostalgia','calmness','power','joyful_activation','tension','sadness','mood','liked','disliked','age','gender','mother tongue'],axis=1)
rowdata_pop.tail(10)

,track id,genre,amazement_rank,solemnity_rank,tenderness_rank,nostalgia_rank,calmness_rank,power_rank,joyful_activation_rank,tension_rank,sadness_rank
8288,391,pop,0.250000,0.000000,0.166667,0.166667,0.000000,0.250000,0.750000,0.166667,0.083333
8300,392,pop,0.083333,0.000000,0.583333,0.416667,0.666667,0.000000,0.083333,0.000000,0.333333
8312,393,pop,0.166667,0.500000,0.250000,0.333333,0.500000,0.000000,0.166667,0.333333,0.166667
8324,394,pop,0.230769,0.076923,0.000000,0.230769,0.000000,0.538462,0.538462,0.384615,0.230769
8337,395,pop,0.500000,0.000000,0.166667,0.166667,0.083333,0.083333,0.750000,0.083333,0.000000
8349,396,pop,0.250000,0.166667,0.250000,0.500000,0.333333,0.000000,0.166667,0.083333,0.166667
8361,397,pop,0.090909,0.090909,0.090909,0.181818,0.181818,0.363636,0.000000,0.363636,0.545455
8372,398,pop,0.272727,0.000000,0.000000,0.363636,0.272727,0.181818,0.636364,0.272727,0.181818
8383,399,pop,0.333333,0.083333,0.583333,0.583333,0.666667,0.083333,0.166667,0.000000,0.250000
8395,400,pop,0.250000,0.166667,0.083333,0.333333,0.083333,0.000000,0.250000,0.500000,0.416667


In [56]:
def recommender(song_type,song_emotion):
    if(song_type=="classical"):
        if(song_emotion==0):
            result=rowdata_classical.sort_values(by=["amazement_rank"],ascending=False)
            return result
        if(song_emotion==1):
            result=rowdata_classical.sort_values(by=["solemnity_rank"],ascending=False)
            return result
        if(song_emotion==2):
            result=rowdata_classical.sort_values(by=["tenderness_rank"],ascending=False)
            return result
        if(song_emotion==3):
            result=rowdata_classical.sort_values(by=["nostalgia_rank"],ascending=False)
            return result
        if(song_emotion==4):
            result=rowdata_classical.sort_values(by=["calmness_rank"],ascending=False)
            return result
        if(song_emotion==5):
            result=rowdata_classical.sort_values(by=["power_rank"],ascending=False)
            return result
        if(song_emotion==6):
            result=rowdata_classical.sort_values(by=["joyful_activation_rank"],ascending=False)
            return result
        if(song_emotion==7):
            result=rowdata_classical.sort_values(by=["tension_rank"],ascending=False)
            return result
        if(song_emotion==8):
            result=rowdata_classical.sort_values(by=["sadness_rank"],ascending=False)
            return result
        
        
        
    elif(song_type=="rock"):
        if(song_emotion==0):
            result=rowdata_rock.sort_values(by=["amazement_rank"],ascending=False)
            return result
        if(song_emotion==1):
            result=rowdata_rock.sort_values(by=["solemnity_rank"],ascending=False)
            return result
        if(song_emotion==2):
            result=rowdata_rock.sort_values(by=["tenderness_rank"],ascending=False)
            return result
        if(song_emotion==3):
            result=rowdata_rock.sort_values(by=["nostalgia_rank"],ascending=False)
            return result
        if(song_emotion==4):
            result=rowdata_rock.sort_values(by=["calmness_rank"],ascending=False)
            return result
        if(song_emotion==5):
            result=rowdata_rock.sort_values(by=["power_rank"],ascending=False)
            return result
        if(song_emotion==6):
            result=rowdata_rock.sort_values(by=["joyful_activation_rank"],ascending=False)
            return result
        if(song_emotion==7):
            result=rowdata_rock.sort_values(by=["tension_rank"],ascending=False)
            return result
        if(song_emotion==8):
            result=rowdata_rock.sort_values(by=["sadness_rank"],ascending=False)
            return result
    
    elif(song_type=="electronic"):
        if(song_emotion==0):
            result=rowdata_electronic.sort_values(by=["amazement_rank"],ascending=False)
            return result
        if(song_emotion==1):
            result=rowdata_electronic.sort_values(by=["solemnity_rank"],ascending=False)
            return result
        if(song_emotion==2):
            result=rowdata_electronic.sort_values(by=["tenderness_rank"],ascending=False)
            return result
        if(song_emotion==3):
            result=rowdata_electronic.sort_values(by=["nostalgia_rank"],ascending=False)
            return result
        if(song_emotion==4):
            result=rowdata_electronic.sort_values(by=["calmness_rank"],ascending=False)
            return result
        if(song_emotion==5):
            result=rowdata_electronic.sort_values(by=["power_rank"],ascending=False)
            return result
        if(song_emotion==6):
            result=rowdata_electronic.sort_values(by=["joyful_activation_rank"],ascending=False)
            return result
        if(song_emotion==7):
            result=rowdata_electronic.sort_values(by=["tension_rank"],ascending=False)
            return result
        if(song_emotion==8):
            result=rowdata_electronic.sort_values(by=["sadness_rank"],ascending=False)
            return result
    elif(song_type=="pop"):
        if(song_emotion==0):
            result=rowdata_pop.sort_values(by=["amazement_rank"],ascending=False)
            return result
        if(song_emotion==1):
            result=rowdata_pop.sort_values(by=["solemnity_rank"],ascending=False)
            return result
        if(song_emotion==2):
            result=rowdata_pop.sort_values(by=["tenderness_rank"],ascending=False)
            return result
        if(song_emotion==3):
            result=rowdata_pop.sort_values(by=["nostalgia_rank"],ascending=False)
            return result
        if(song_emotion==4):
            result=rowdata_pop.sort_values(by=["calmness_rank"],ascending=False)
            return result
        if(song_emotion==5):
            result=rowdata_pop.sort_values(by=["power_rank"],ascending=False)
            return result
        if(song_emotion==6):
            result=rowdata_pop.sort_values(by=["joyful_activation_rank"],ascending=False)
            return result
        if(song_emotion==7):
            result=rowdata_pop.sort_values(by=["tension_rank"],ascending=False)
            return result
        if(song_emotion==8):
            result=rowdata_pop.sort_values(by=["sadness_rank"],ascending=False)
            return result
        
        
            

In [63]:
if(emotion_dict[maxindex]=="Angry"):
    song_emotion=7
elif(emotion_dict[maxindex]=="Disgust"):
    song_emotion=7
elif(emotion_dict[maxindex]=="Fear"):
    song_emotion=7
elif(emotion_dict[maxindex]=="Happy"):
    song_emotion=0
elif(emotion_dict[maxindex]=="Sad"):
    song_emotion=8
elif(emotion_dict[maxindex]=="Surprise"):
    song_emotion=1
elif(emotion_dict[maxindex]=="Neutral"):
    song_emotion=4

In [65]:
result=recommender("classical",song_emotion)
result=result.head(10)
print(result)
result=result.head(1)
for i in result["track id"]:
    song=str(i)+"."+"mp3"
    print("yoursong")
    playsound(song)
    
'''result=recommender("rock",song_emotion)
result=result.head(10)
print(result)
result=result.head(1)
for i in result["track id"]:
    song=str(i)+"."+"mp3"
    print("yoursong")
    playsound(song)
    
result=recommender("electronic",song_emotion)
result=result.head(10)
print(result)
result=result.head(1)
for i in result["track id"]:
    song=str(i)+"."+"mp3"
    print("yoursong")
    playsound(song)
    
result=recommender("pop",song_emotion)
result=result.head(10)
print(result)
result=result.head(1)
for i in result["track id"]:
    song=str(i)+"."+"mp3"
    print("yoursong")
    playsound(song)'''

      track id      genre  amazement_rank  solemnity_rank  tenderness_rank  \
1600        39  classical        0.526316        0.263158         0.000000   
1334        30  classical        0.478261        0.369565         0.021739   
779         18  classical        0.372549        0.176471         0.058824   
2415        85  classical        0.352941        0.058824         0.058824   
274          7  classical        0.340426        0.085106         0.148936   
505         12  classical        0.313725        0.058824         0.156863   
1232        28  classical        0.296296        0.074074         0.240741   
1791        50  classical        0.294118        0.352941         0.176471   
2586        95  classical        0.294118        0.176471         0.235294   
2569        94  classical        0.294118        0.588235         0.000000   

      nostalgia_rank  calmness_rank  power_rank  joyful_activation_rank  \
1600        0.000000       0.000000    0.473684                0.6

'result=recommender("rock",song_emotion)\nresult=result.head(10)\nprint(result)\nresult=result.head(1)\nfor i in result["track id"]:\n    song=str(i)+"."+"mp3"\n    print("yoursong")\n    playsound(song)\n    \nresult=recommender("electronic",song_emotion)\nresult=result.head(10)\nprint(result)\nresult=result.head(1)\nfor i in result["track id"]:\n    song=str(i)+"."+"mp3"\n    print("yoursong")\n    playsound(song)\n    \nresult=recommender("pop",song_emotion)\nresult=result.head(10)\nprint(result)\nresult=result.head(1)\nfor i in result["track id"]:\n    song=str(i)+"."+"mp3"\n    print("yoursong")\n    playsound(song)'